<h1><center>Credit Risk Analysis using IBM Snap ML</center></h1>

# Download Input Dataset

In [1]:
# Many times better training time with snapML compared to sklearn with this full dataset
!wget -O credit_customer_history.csv -nc https://ibm.box.com/shared/static/lkvsjt6cch2fdtecobhjf54snmm6eagt.csv

# Download reduced dataset
#!wget -O credit_customer_history.csv https://ibm.box.com/shared/static/gucqudgok3o0awc0o1vfhbo2ix2k0wqa.csv

File `credit_customer_history.csv' already there; not retrieving.


# Imports

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score
from scipy.stats import chi2_contingency,ttest_ind
from sklearn.utils import shuffle
import time

import warnings
warnings.filterwarnings('ignore')


/home/zrlaan/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Dataset Visualization

In [4]:
cust_pd_full = pd.read_csv('credit_customer_history.csv')

rows=1000000
cust_pd = cust_pd_full.head(rows)
print("There are " + str(len(cust_pd_full)) + " observations in the customer history dataset.")
print("There are " + str(len(cust_pd_full.columns)) + " variables in the dataset.")

cust_pd.head()

There are 10000000 observations in the customer history dataset.
There are 19 variables in the dataset.


EMI_TENURE              CREDIT_HISTORY TRANSACTION_CATEGORY  \
0          77  EXISTING CREDITS PAID BACK            EDUCATION   
1         119  EXISTING CREDITS PAID BACK          ELECTRONICS   
2          84  EXISTING CREDITS PAID BACK            FURNITURE   
3         119               DELAY IN PAST            FURNITURE   
4         105               DELAY IN PAST            FURNITURE   

   TRANSACTION_AMOUNT      ACCOUNT_TYPE  ACCOUNT_AGE STATE IS_URBAN  \
0               27630      UNKNOWN/NONE  above 7 YRS    CT       NO   
1               31314  above 1000 K USD   4 to 7 YRS    CT      YES   
2               27630  above 1000 K USD   4 to 7 YRS    PA       NO   
3               33156  above 1000 K USD   up to 1 YR    PA      YES   
4               23946  above 1000 K USD   up to 1 YR    CT       NO   

  IS_STATE_BORDER HAS_CO_APPLICANT HAS_GUARANTOR OWN_REAL_ESTATE  \
0             YES              YES           YES              NO   
1             YES              YES           YES              NO   
2              NO              YES           YES             YES   
3              NO              YES            NO              NO   
4             YES              YES           YES             YES   

  OTHER_INSTALMENT_PLAN OWN_RESIDENCE  NUMBER_CREDITS  RFM_SCORE OWN_CAR  \
0                   YES            NO               0          4      NO   
1                    NO           YES               0          3     YES   
2                    NO           YES               0          3     YES   
3                    NO           YES               0          3      NO   
4                   YES            NO               0          3     YES   

  SHIP_INTERNATIONAL IS_DEFAULT  
0                 NO         No  
1                YES         No  
2                YES         No  
3                 NO        Yes  
4                YES         No

# Data Preprocessing

In [5]:
# Split dataframe into Features (X) and Labels (y)
cust_pd_Y = cust_pd_full[['IS_DEFAULT']]
cust_pd_X = cust_pd_full.drop(['IS_DEFAULT'],axis=1)

print('cust_pd_X.shape=', cust_pd_X.shape, 'cust_pd_Y.shape=', cust_pd_Y.shape)

cust_pd_X.shape= (10000000, 18) cust_pd_Y.shape= (10000000, 1)


# Transform Labels (y)

In [6]:
cust_pd_Y.head()

IS_DEFAULT
0         No
1         No
2         No
3        Yes
4         No

In [7]:
le = LabelEncoder()
cust_pd_Y['IS_DEFAULT'] = le.fit_transform(cust_pd_Y['IS_DEFAULT'])
cust_pd_Y.head()

IS_DEFAULT
0           0
1           0
2           0
3           1
4           0

# Transform Features (X)

In [8]:
print('features X dataframe shape = ', cust_pd_X.shape)
cust_pd_X.head()

features X dataframe shape =  (10000000, 18)


EMI_TENURE              CREDIT_HISTORY TRANSACTION_CATEGORY  \
0          77  EXISTING CREDITS PAID BACK            EDUCATION   
1         119  EXISTING CREDITS PAID BACK          ELECTRONICS   
2          84  EXISTING CREDITS PAID BACK            FURNITURE   
3         119               DELAY IN PAST            FURNITURE   
4         105               DELAY IN PAST            FURNITURE   

   TRANSACTION_AMOUNT      ACCOUNT_TYPE  ACCOUNT_AGE STATE IS_URBAN  \
0               27630      UNKNOWN/NONE  above 7 YRS    CT       NO   
1               31314  above 1000 K USD   4 to 7 YRS    CT      YES   
2               27630  above 1000 K USD   4 to 7 YRS    PA       NO   
3               33156  above 1000 K USD   up to 1 YR    PA      YES   
4               23946  above 1000 K USD   up to 1 YR    CT       NO   

  IS_STATE_BORDER HAS_CO_APPLICANT HAS_GUARANTOR OWN_REAL_ESTATE  \
0             YES              YES           YES              NO   
1             YES              YES           YES              NO   
2              NO              YES           YES             YES   
3              NO              YES            NO              NO   
4             YES              YES           YES             YES   

  OTHER_INSTALMENT_PLAN OWN_RESIDENCE  NUMBER_CREDITS  RFM_SCORE OWN_CAR  \
0                   YES            NO               0          4      NO   
1                    NO           YES               0          3     YES   
2                    NO           YES               0          3     YES   
3                    NO           YES               0          3      NO   
4                   YES            NO               0          3     YES   

  SHIP_INTERNATIONAL  
0                 NO  
1                YES  
2                YES  
3                 NO  
4                YES

# One-Hot Encoding of Categorical Features

In [9]:
categoricalColumns = ['CREDIT_HISTORY', 'TRANSACTION_CATEGORY', 'ACCOUNT_TYPE', 'ACCOUNT_AGE',
                      'STATE', 'IS_URBAN', 'IS_STATE_BORDER', 'HAS_CO_APPLICANT', 'HAS_GUARANTOR',
                      'OWN_REAL_ESTATE', 'OTHER_INSTALMENT_PLAN',
                      'OWN_RESIDENCE', 'RFM_SCORE', 'OWN_CAR', 'SHIP_INTERNATIONAL']
cust_pd_X = pd.get_dummies(cust_pd_X, columns=categoricalColumns)
cust_pd_X.head()

print('features X dataframe shape = ', cust_pd_X.shape)

features X dataframe shape =  (10000000, 51)


# Normalize Features

In [10]:
min_max_scaler = MinMaxScaler()
features = min_max_scaler.fit_transform(cust_pd_X)
features = normalize(features, axis=1, norm='l1')

cust_pd_X = pd.DataFrame(features,columns=cust_pd_X.columns)
cust_pd_X.head()

EMI_TENURE  TRANSACTION_AMOUNT  NUMBER_CREDITS  \
0    0.027542            0.033533             0.0   
1    0.041751            0.037277             0.0   
2    0.029971            0.033449             0.0   
3    0.041659            0.039384             0.0   
4    0.037350            0.028902             0.0   

   CREDIT_HISTORY_ALL CREDITS PAID BACK  CREDIT_HISTORY_CRITICAL ACCOUNT  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   CREDIT_HISTORY_DELAY IN PAST  CREDIT_HISTORY_EXISTING CREDITS PAID BACK  \
0                      0.000000                                   0.062595   
1                      0.000000                                   0.061398   
2                      0.000000                                   0.062439   
3                      0.061264                                   0.000000   
4                      0.062250                                   0.000000   

   CREDIT_HISTORY_NONE TAKEN  TRANSACTION_CATEGORY_EDUCATION  \
0                        0.0                        0.062595   
1                        0.0                        0.000000   
2                        0.0                        0.000000   
3                        0.0                        0.000000   
4                        0.0                        0.000000   

   TRANSACTION_CATEGORY_ELECTRONICS  TRANSACTION_CATEGORY_FURNITURE  \
0                          0.000000                        0.000000   
1                          0.061398                        0.000000   
2                          0.000000                        0.062439   
3                          0.000000                        0.061264   
4                          0.000000                        0.062250   

   TRANSACTION_CATEGORY_NEW CAR  TRANSACTION_CATEGORY_OTHER  \
0                           0.0                         0.0   
1                           0.0                         0.0   
2                           0.0                         0.0   
3                           0.0                         0.0   
4                           0.0                         0.0   

   TRANSACTION_CATEGORY_RETRAINING  TRANSACTION_CATEGORY_USED CAR  \
0                              0.0                            0.0   
1                              0.0                            0.0   
2                              0.0                            0.0   
3                              0.0                            0.0   
4                              0.0                            0.0   

   ACCOUNT_TYPE_100 to 500 K USD  ACCOUNT_TYPE_500 to 1000 K USD  \
0                            0.0                             0.0   
1                            0.0                             0.0   
2                            0.0                             0.0   
3                            0.0                             0.0   
4                            0.0                             0.0   

   ACCOUNT_TYPE_UNKNOWN/NONE  ACCOUNT_TYPE_above 1000 K USD  \
0                   0.062595                       0.000000   
1                   0.000000                       0.061398   
2                   0.000000                       0.062439   
3                   0.000000                       0.061264   
4                   0.000000                       0.062250   

   ACCOUNT_TYPE_up to 100 K USD  ACCOUNT_AGE_1 to 4 YRS  \
0                           0.0                     0.0   
1                           0.0                     0.0   
2                           0.0                     0.0   
3                           0.0                     0.0   
4                           0.0                     0.0   

   ACCOUN

# Generate Train and Test Datasets

In [11]:
labels    = cust_pd_Y.values
features  = cust_pd_X.values

labels = np.reshape(labels,(-1,1))

X_train,X_test,y_train,y_test = \
       train_test_split(features, labels, test_size=0.3, random_state=42, stratify=labels)
                    
print('X_train.shape=', X_train.shape, 'Y_train.shape=', y_train.shape)
print('X_test.shape=', X_test.shape, 'Y_test.shape=', y_test.shape)

X_train.shape= (7000000, 51) Y_train.shape= (7000000, 1)
X_test.shape= (3000000, 51) Y_test.shape= (3000000, 1)


# Train a Logistic Regression Model using Scikit-Learn

In [26]:
#from sklearn.linear_model import LogisticRegression
#sklearn_lr = LogisticRegression()

from pai4sk.linear_model import LogisticRegression
sklearn_lr = LogisticRegression(solver='liblinear') 
print(sklearn_lr)

LogisticRegression(C=1.0, batch_size=100, class_weight=None, device_ids=[],
          dual=False, eta=0.3, fit_intercept=True, grad_clip=1,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, num_threads=1, penalty='l2', privacy=False,
          privacy_epsilon=10, random_state=None,
          return_training_history=None, solver='liblinear', tol=0.0001,
          use_gpu=True, verbose=0, warm_start=False)


In [27]:
# Train a logistic regression model using Scikit-Learn
t0 = time.time()
sklearn_lr.fit(X_train, y_train)
sklearn_time = time.time() - t0
print("[sklearn] Training time (s):  {0:.2f}".format(sklearn_time))

# Evaluate accuracy on test set
sklearn_pred = sklearn_lr.predict(X_test)
print('[sklearn] Accuracy score : {0:.6f}'.format(accuracy_score(y_test, sklearn_pred)))

[sklearn] Training time (s):  80.02
[sklearn] Accuracy score : 0.959830


# Train a Logistic Regression Model using Snap ML

In [32]:
from pai4sk import LogisticRegression
snapml_lr = LogisticRegression(use_gpu=True, device_ids=[0,1])
print(snapml_lr.get_params())

[Info] If set num_threads should be a multiple of 32. GPU training will run with num_threads=256.
{'max_iter': 1000, 'regularizer': 1.0, 'use_gpu': True, 'device_ids': array([0, 1], dtype=uint32), 'class_weight': None, 'dual': True, 'verbose': False, 'num_threads': 256, 'penalty': 'l2', 'tol': 0.001, 'return_training_history': None, 'privacy': False, 'eta': 0.3, 'batch_size': 100, 'grad_clip': 1, 'privacy_epsilon': 10, 'fit_intercept': False, 'intercept_scaling': 1.0}


In [34]:
# Train a logistic regression model using Snap ML
t0 = time.time()
model = snapml_lr.fit(X_train, y_train)
snapml_time = time.time() - t0
print("[Snap ML] Training time (s):  {0:.2f}".format(snapml_time))

# Evaluate accuracy on test set
snapml_pred = snapml_lr.predict(X_test)
print('[Snap ML] Accuracy score : {0:.6f}'.format(accuracy_score(y_test, snapml_pred)))
print('[Logistic Regression] Snap ML vs. sklearn speedup : {0:.2f}x '.format(sklearn_time/snapml_time))

[Snap ML] Training time (s):  1.62
[Snap ML] Accuracy score : 0.959837
[Logistic Regression] Snap ML vs. sklearn speedup : 49.35x 


&copy; Copyright IBM Corporation 2019